In [1]:
#Importamos las librerias
import pandas as pd
import numpy as np

In [2]:
#Cargamos los archivos que tenemos que usar
games = pd.read_csv("Csv_limpios/steam_games_limpio.csv")
items = pd.read_csv("Csv_limpios/steam_items_limpio.csv")
reviews = pd.read_csv("Csv_limpios/steam_reviews_limpio.csv")

In [3]:
#Renombramos la columna id por item_id para poder usar el merge
games.rename(columns={"id" : "item_id"}, inplace=True)

In [4]:
#Seleccionamos las columnas que vamos a usar
juegos = games[["item_id", "title"]]

In [5]:
#Eliminamos los duplicados
juegos = juegos.drop_duplicates().reset_index(drop=True)

In [6]:
juegos

,item_id,title
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,772540,Battle Royale Trainer
...,...,...
30034,745400,Kebab it Up!
30035,773640,Colony On Mars
30036,733530,LOGistICAL: South Africa
30037,610660,Russian Roads


In [7]:
#Unimos los dataframes por su item_id
juegos_reviews = pd.merge(reviews, juegos, how="inner", on="item_id")
#Dejamos los juegos que tengan valoracion negativa y no sean recomendados
juegos_reviews = juegos_reviews[(juegos_reviews["sentiment_analysis"].isin([0])) & (juegos_reviews["recommend"] == 0)].reset_index(drop=True)
#Eliminamos las columnas que no se necesitan
juegos_reviews.drop(columns=["item_id", "user_id"], inplace=True)

In [8]:
#Unimos los df por su item_id
juegos_positivo= pd.merge(reviews, juegos, how="inner", on="item_id")
#Eliminamos las columnas que no se necesitan
juegos_positivo.drop(columns=["item_id", "user_id"], inplace=True)
#Dejamos los juegos que tengan una valoracion positiva y sean recomendados
juegos_positivo = juegos_positivo[(juegos_positivo["sentiment_analysis"].isin([1, 2])) & (juegos_positivo["recommend"] == 1)].reset_index(drop=True)

In [9]:
juegos_positivo

,recommend,year,sentiment_analysis,title
0,1,2011.0,2,Killing Floor
1,1,2015.0,2,Killing Floor
2,1,2015.0,1,Killing Floor
3,1,2014.0,1,Killing Floor
4,1,2010.0,2,Killing Floor
...,...,...,...,...
38528,1,2015.0,2,Asteria
38529,1,2015.0,2,Street Fighter X Tekken
38530,1,2015.0,1,The Journey Down: Chapter One
38531,1,2015.0,1,The Journey Down: Chapter Two


In [10]:
#Pasamos los años a tipo INT
juegos_reviews["year"] = juegos_reviews["year"].astype(int)
juegos_positivo["year"] = juegos_positivo["year"].astype(int)

#Pasamos los df a csv
juegos_reviews.to_csv("Csv_api/funcion4.csv", index=False)
#juegos_positivo = juegos_positivo.sample(n=20000, random_state=123)
juegos_positivo.to_csv("Csv_api/funcion4_2.csv", index=False)


In [11]:
#def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 
#(reviews.recommend = False y comentarios negativos)
def UsersNotRecommend(año: int):
    if 2010 <= año <= 2015:
        # Filtramos los DataFrames por el año ingresado
        filtro = juegos_reviews[juegos_reviews["year"] == año]
        filtro2 = juegos_positivo[juegos_positivo["year"] == año]

        # Agrupamos por el título de los juegos y contamos la cantidad de reseñas negativas que tiene cada juego
        negativos = filtro.groupby("title")["sentiment_analysis"].count().reset_index(name="cantidad_reseñas_n")
        positivos = filtro2.groupby("title")["sentiment_analysis"].count().reset_index(name="cantidad_reseñas_p")

        # Reindexar ambos DataFrames por 'title'
        negativos = negativos.set_index('title')
        positivos = positivos.set_index('title')

        # Crear una copia independiente de negativos
        resta = negativos.copy()

        # Realizar la resta
        resta["cantidad_reseñas"] = positivos["cantidad_reseñas_p"] - resta["cantidad_reseñas_n"]

        # Restaurar el índice si es necesario
        resta.reset_index(inplace=True)

        # Ordenar y obtener el top 3
        top3 = resta.sort_values(by="cantidad_reseñas", ascending=True).head(3)

        # Devolver el resultado en formato JSON
        return top3.to_json(orient="records")
    else:
        return "Por favor, ingrese un año válido entre 2011 y 2015."

UsersNotRecommend(2013)

'[{"title":"Half-Life: Before","cantidad_rese\\u00f1as_n":5,"cantidad_rese\\u00f1as":-2.0},{"title":"A.V.A. Alliance of Valiant Arms\\u2122","cantidad_rese\\u00f1as_n":1,"cantidad_rese\\u00f1as":0.0},{"title":"Marvel Puzzle Quest","cantidad_rese\\u00f1as_n":1,"cantidad_rese\\u00f1as":0.0}]'